## データセットのダウンロード

本ハンズオンでは[FER-2013](https://www.kaggle.com/datasets/msambare/fer2013/data)というデータセットを使用します。

本ハンズオンのレポジトリ（[kaira_tutorial2025](https://github.com/kazumasa-okamoto/kaira_tutorial2025)）からダウンロードするか、もしくはKaggleのアカウントを持っている方はkaggleからダウンロードしてください

### レポジトリからダウンロード

In [ ]:
! wget https://github.com/kazumasa-okamoto/kaira_tutorial2025/blob/main/fer2013.zip

In [ ]:
! unzip fer2013.zip

###  Kaggleからダウンロード

本ハンズオンでは[FER-2013](https://www.kaggle.com/datasets/msambare/fer2013/data)というデータセットをKaggleからダウンロードして使用します。

したがって、そのためにはKaggleのアカウントが必要になるので、[Kaggle](https://www.kaggle.com/)からアカウントを作成してください。

In [ ]:
! pip install kaggle

ここではKaggleのAPIを利用します。
サイトの右上にある自分のアイコンをクリックし、SettingsからCreate New Tokenをクリックし、kaggle.jsonをダウンロードしてください。
そして、ダウンロードしたkaggle.jsonを下のセルでアップロードしてください。



In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

これでKaggleのAPIが利用できるようになりました。データセットをダウンロードして、解凍します。

In [ ]:
! kaggle datasets download msambare/fer2013

In [ ]:
! unzip fer2013.zip

## データセットの作成

本ハンズオンでは[Pytorch](https://pytorch.org/)というライブラリを使用します。

ダウンロードしたデータセットをPytorchで扱える形式に変換していきます。

In [ ]:
# ライブラリのインポート
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader

PytorchではTensorという形式を利用するのでこれに変換します。

またニューラルネットワークの学習を行う際には、データの正規化を施した方が良いのでそれも行います。

ここでは画像の画素値（0～255）を０0～1の値に変換することで、ニューラルネットワークにとって学習しやすくさせています。

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    # データのTensor化
    transforms.ToTensor(),
    # データの正規化
    transforms.Normalize([0.5], [0.5])
])

In [ ]:
train_dataset= datasets.ImageFolder(root='/content/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/test', transform=transform)

PytorchではDataLoaderによって、データがモデルへと引き渡されます。

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

今回学習に使用するデータの中身を見てみましょう。

In [ ]:
import matplotlib.pyplot as plt

images, labels = next(iter(train_loader))

images = images * 0.5 + 0.5
class_names = train_dataset.classes


# プロット設定
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Images from Training Set', fontsize=18)

# 画像表示
for i, ax in enumerate(axes.flat):
    img = images[i].squeeze(0)
    ax.imshow(img, cmap='gray')
    ax.set_title(class_names[labels[i]], fontsize=10)
    ax.axis('off')

plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

ラベルの分布も確認しておきましょう。

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# クラス名（ImageFolderのクラス順に対応）
class_names = train_dataset.classes

# ラベルをカウント
train_counts = Counter(train_dataset.targets)
test_counts = Counter(test_dataset.targets)

# クラス順に整列させた数値リスト
train_values = [train_counts[i] for i in range(len(class_names))]
test_values = [test_counts[i] for i in range(len(class_names))]

# プロット
x = range(len(class_names))
width = 0.4  # 棒グラフの幅（ずらして重ならないようにもできる）

plt.figure(figsize=(9, 6))
plt.bar(x, train_values, width=width, alpha=0.6, label='Train', color='royalblue')
plt.bar(x, test_values, width=width, alpha=0.6, label='Test', color='orangered')

plt.xticks(ticks=x, labels=class_names, rotation=45)
plt.title('Class Distribution in FER2013 (Train vs Test)', fontsize=15)
plt.xlabel('Emotion Class')
plt.ylabel('Number of Images')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## 乱数の固定

機械学習には確率的な操作が多く含まれます。乱数を固定することで実験結果の再現性が高まります。

In [ ]:
# ライブラリのインポート
import numpy as np
import torch

In [ ]:
def fix_seed(seed=123):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms = True

In [ ]:
fix_seed()

## ニューラルネットワーク

In [ ]:
# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

学習一般に用いる関数は[『
最短コースでわかる PyTorch ＆深層学習プログラミング』](https://bookplus.nikkei.com/atcl/catalog/21/283980/)という本を参考にしています。

ニューラルネットワークの学習では、予測の答えと本当の答えのズレを測定するために損失関数というものを用います。

この損失が小さくなるように、ニューラルネットワークは少しずつ調整されていきます。

損失を計算する用の関数、eval_lossを定義しましょう。

In [ ]:
# 損失計算用
def eval_loss(loader, device, model, criterion):

    # データローダーから最初の1セットを取得する
    for images, labels in loader:
        break

    # デバイスの割り当て
    inputs = images.to(device)
    labels = labels.to(device)

    # 予測計算
    outputs = model(inputs)

    #  損失計算
    loss = criterion(outputs, labels)

    return loss

ニューラルネットワークの学習は、予測計算・損失計算・勾配計算・パラメータの修正の繰り返しで進んでいきます。

どのようにパラメータを修正すれば、最も損失が小さくなるかを計算しているわけです。

学習用の関数fitを定義しましょう。

In [ ]:
# 学習用関数
def fit(model, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)

    for epoch in range(base_epochs, num_epochs+base_epochs):
        # 1エポックあたりの正解数(精度計算用)
        n_train_acc, n_val_acc = 0, 0
        # 1エポックあたりの累積損失(平均化前)
        train_loss, val_loss = 0, 0
        # 1エポックあたりのデータ累積件数
        n_train, n_test = 0, 0

        #訓練フェーズ
        model.train()

        for inputs, labels in tqdm(train_loader):
            # 1バッチあたりのデータ件数
            train_batch_size = len(labels)
            # 1エポックあたりのデータ累積件数
            n_train += train_batch_size

            # GPUヘ転送
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            optimizer.zero_grad()
            # 予測計算
            outputs = model(inputs)
            # 損失計算
            loss = criterion(outputs, labels)
            # 勾配計算
            loss.backward()
            # パラメータ修正
            optimizer.step()

            # 予測ラベル導出
            predicted = torch.max(outputs, 1)[1]

            # 平均前の損失と正解数の計算
            # lossは平均計算が行われているので平均前の損失に戻して加算
            train_loss += loss.item() * train_batch_size
            n_train_acc += (predicted == labels).sum().item()

        #予測フェーズ
        model.eval()

        for inputs_test, labels_test in test_loader:
            # 1バッチあたりのデータ件数
            test_batch_size = len(labels_test)
            # 1エポックあたりのデータ累積件数
            n_test += test_batch_size

            # GPUヘ転送
            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            # 予測計算
            outputs_test = model(inputs_test)
            # 損失計算
            loss_test = criterion(outputs_test, labels_test)

            # 予測ラベル導出
            predicted_test = torch.max(outputs_test, 1)[1]

            #  平均前の損失と正解数の計算
            # lossは平均計算が行われているので平均前の損失に戻して加算
            val_loss +=  loss_test.item() * test_batch_size
            n_val_acc +=  (predicted_test == labels_test).sum().item()

        # 精度計算
        train_acc = n_train_acc / n_train
        val_acc = n_val_acc / n_test
        # 損失計算
        avg_train_loss = train_loss / n_train
        avg_val_loss = val_loss / n_test
        # 結果表示
        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {val_acc:.5f}')
        # 記録
        item = np.array([epoch+1, avg_train_loss, train_acc, avg_val_loss, val_acc])
        history = np.vstack((history, item))
    return history

学習ログを解析する用の関数、evaluate_historyも定義しておきましょう。

学習を繰り返していくごとに、損失や精度がどのように変化していくかを見ていきます。


In [ ]:
# 学習ログ解析
plt.rcParams.update({
    'font.size': 12,
    'axes.edgecolor': '#333333',
    'axes.labelcolor': '#333333',
    'xtick.color': '#333333',
    'ytick.color': '#333333',
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'figure.figsize': (10, 6),
    'lines.linewidth': 2,
    'grid.color': '#cccccc',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
    'legend.frameon': False
})

def evaluate_history(history):
    # 損失と精度の確認
    print(f'Initial: Loss = {history[0,3]:.5f}, Accuracy = {history[0,4]:.5f}')
    print(f'Final:   Loss = {history[-1,3]:.5f}, Accuracy = {history[-1,4]:.5f}')

    num_epochs = len(history)
    unit = num_epochs / 10

    # 学習曲線（損失）
    plt.figure(figsize=(10, 6))
    plt.plot(history[:,0], history[:,1], label='Training', color='#1f77b4', linewidth=2)
    plt.plot(history[:,0], history[:,3], label='Validation', color='#ff7f0e', linewidth=2)
    plt.xticks(np.arange(0, num_epochs+1, unit))
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # 学習曲線（精度）
    plt.figure(figsize=(10, 6))
    plt.plot(history[:,0], history[:,2], label='Training', color='#1f77b4', linewidth=2)
    plt.plot(history[:,0], history[:,4], label='Validation', color='#ff7f0e', linewidth=2)
    plt.xticks(np.arange(0, num_epochs+1, unit))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

Pytorchではモデルを次のようにして定義します。

今回のモデルは次の要素から構成されています。
- nn.Flatten：画像という2Dのデータを1列に並べて、ニューラルネットワークに通せる形に変換する
- nn.Linear：各入力に重みをかけて合計し、次の層へと伝える
- nn.ReLU：活性化関数。これによってニューラルネットワークは表現力を獲得する
- nn.BatchNorm1d・nn.Dropout：学習を安定化させる効果がある

In [ ]:
class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=256, dropout=0.5):
        super().__init__()
        self.flatten = nn.Flatten()

        self.l1 = nn.Linear(n_input, n_hidden)
        self.bn1 = nn.BatchNorm1d(n_hidden)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)

        self.l2 = nn.Linear(n_hidden, n_hidden // 2)
        self.bn2 = nn.BatchNorm1d(n_hidden // 2)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout)

        self.out = nn.Linear(n_hidden // 2, n_output)

    def forward(self, x):
        x = self.flatten(x)

        x = self.l1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)

        x = self.l2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)

        x = self.out(x)
        return x

In [ ]:
# 入力次元数（今回は48×48のモノクロ画像なので2304）
n_input = 2304
# 出力次元数（分類先クラス数：今回は7）
n_output = 7
#   隠れ層のノード数
n_hidden = 128

# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# モデルインスタンス生成
model = Net(n_input, n_output, n_hidden).to( device)

# 損失関数
criterion = nn.CrossEntropyLoss()
# 学習率
lr = 1e-2
# 最適化関数
optimizer = optim.SGD(model.parameters(), lr=lr)
# 繰り返し回数
num_epochs = 50

# 評価結果記録用
history = np.zeros((0,5))

# 学習
history = fit(model, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
evaluate_history(history)

モデルがどのように分類を間違えているか、混同行列を用いて確認してみましょう。

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def evaluate_with_confusion_matrix(model, dataloader, device):
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 混同行列の計算
    cm = confusion_matrix(all_labels, all_preds)

    # クラス名の取得
    class_names = dataloader.dataset.classes

    # 混同行列の表示
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot()
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
evaluate_with_confusion_matrix(model, test_loader, device)

モデルが間違えている事例を実際に確認してみましょう

In [ ]:
import random

def show_misclassified_images(model, dataloader, device, max_images=9):
    model.eval()
    model.to(device)

    misclassified = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            for img, pred, true in zip(inputs, preds, labels):
                if pred != true:
                    misclassified.append((img.cpu(), pred.item(), true.item()))

    sampled = random.sample(misclassified, min(max_images, len(misclassified)))

    # クラス名の取得
    class_names = dataloader.dataset.classes

    # プロット
    n = len(sampled)
    cols = min(3, n)
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(cols * 4, rows * 4))
    for i, (img, pred, true) in enumerate(sampled):
        plt.subplot(rows, cols, i + 1)
        img = img.squeeze(0).numpy()
        plt.imshow(img, cmap='gray')
        plt.title(f"Pred: {class_names[pred]}\nTrue: {class_names[true]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
show_misclassified_images(model, test_loader, device)

## 畳み込みニューラルネットワーク（CNN）

モデルのアーキテクチャは[Human Emotion Detection 🙂😞😠](https://www.kaggle.com/code/mohamedchahed/human-emotion-detection)のNotebookを参考にしています。

nn.Conv2d：畳み込み層を導入することで、画像におけるピクセルの位置関係を考慮することが可能になりました。

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=7):
        super(CNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),

            nn.Conv2d(32, 64, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 128, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(256),

            nn.Conv2d(256, 256, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.5)
        )

        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.get_flattened_size(), 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def get_flattened_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 1, 48, 48)
            x = self.conv_block1(x)
            x = self.conv_block2(x)
            x = self.conv_block3(x)
            return x.numel()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.fc_block(x)
        return x

In [ ]:
# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# モデルインスタンス生成
model = CNN().to( device)

# 損失関数
criterion = nn.CrossEntropyLoss()
# 学習率
lr = 1e-4
# 最適化関数
optimizer = optim.Adam(model.parameters(), lr=lr)
# 繰り返し回数
num_epochs = 30

# 評価結果記録用
history2 = np.zeros((0,5))

# 学習
history2 = fit(model, optimizer, criterion, num_epochs, train_loader, test_loader, device, history2)

In [ ]:
evaluate_history(history2)

In [ ]:
evaluate_with_confusion_matrix(model, test_loader, device)

In [ ]:
show_misclassified_images(model, test_loader, device)

## 学習済みモデルのファインチューニング

学習済みのモデルを微調整することで精度を出すこともできます。

ResNetという畳み込みを活用しているモデルを使ってみましょう。

In [ ]:
transform_ft = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [ ]:
train_dataset_ft= datasets.ImageFolder(root='/content/train', transform=transform_ft)
test_dataset_ft = datasets.ImageFolder(root='/content/test', transform=transform_ft)

In [ ]:
train_loader_ft = DataLoader(train_dataset_ft, batch_size=64, shuffle=True)
test_loader_ft = DataLoader(test_dataset_ft, batch_size=64, shuffle=False)

In [ ]:
import timm

model = timm.create_model('resnet18', pretrained=True, in_chans=1, num_classes=7)

model = model.to(device)

In [ ]:
# 損失関数
criterion = nn.CrossEntropyLoss()
# 学習率
lr = 1e-3
# 最適化関数
optimizer = optim.Adam(model.parameters(), lr=lr)
# 繰り返し回数
num_epochs = 3

# 評価結果記録用
history3 = np.zeros((0,5))

# 学習
history3 = fit(model, optimizer, criterion, num_epochs, train_loader_ft, test_loader_ft, device, history3)

In [ ]:
evaluate_history(history3)

In [ ]:
evaluate_with_confusion_matrix(model, test_loader_ft, device)

In [ ]:
show_misclassified_images(model, test_loader_ft, device)